# Trustworthiness Scoring for LangChain

This tutorial shows how to use TLM's trustworthiness score in LangChain.

## Setup

We need LangChain, LangGraph, and OpenAI python libraries for this tutorial. 

In [ ]:
%pip install -qU langchain_core langchain_openai langgraph cleanlab_studio

In [1]:
import os
import logging

from cleanlab_studio import Studio

from langchain_core.callbacks import BaseCallbackHandler
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

## Create a simple chain

This tutorial builds upon [LangChain's chatbot tutorial](https://python.langchain.com/docs/how_to/chatbots_memory/) as a starting point.

Let's first create a simple conversational chain using LangChain's most commonly used objects like `prompt_template`, `graph`, `memory`, and `model`. 

In [2]:
# Define prompt template with placeholder for messages
prompt_template = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="messages"),
    ])

In [3]:
# Define graph for managing state
workflow = StateGraph(state_schema=MessagesState)

# Function that calls the LLM given current state
def call_model(state: MessagesState):
    # Given the current state of messages, prompt the model
    response = model.invoke(state["messages"])
    return {"messages": response}

# Define single node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

Next, we instantiate the LLM assitant. We can then ask questions to this assistant.

In [4]:
model = ChatOpenAI(model="gpt-4o-mini", api_key="<openai_api_key>")

In [5]:
# config ID to capture unique state
config = {"configurable": {"thread_id": "without-tlm"}}

query = "Which is the biggest lake in California?"
input_messages = [HumanMessage(query)]

output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

The biggest lake in California is Lake Tahoe. It is renowned for its stunning clarity, beautiful scenery, and recreational opportunities. Lake Tahoe is situated in the Sierra Nevada mountain range, straddling the border between California and Nevada. It is also one of the largest alpine lakes in North America.


Being a stateful assistant with memory, we can ask follow-up questions.

In [6]:
query = "How deep is it?"
input_messages = [HumanMessage(query)]

output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Lake Tahoe is approximately 1,645 feet (501 meters) deep, making it the second-deepest lake in the United States, after Crater Lake in Oregon. Its significant depth contributes to its striking blue color and clarity.


## Integrating trustworthiness check

It's easy to integrate TLM's trustworthiness score in an existing custom-built chain (using any other LLM generator, streaming or not). 

In this case, we'd define a [Langchain callback](https://python.langchain.com/docs/concepts/callbacks/) that would trigger when the LLM finishes generating response. 
Both the prompt (with system, user, context messages) and the response are used to calculate the score.

Let's define this custom callback handler.

In [7]:
class TrustworthinessScoreCallback(BaseCallbackHandler):
    def __init__(self, tlm, threshold = 0.7, explanation = True):
        self.tlm = tlm
        # Keep track of the prompt
        self.prompt = ""
        # Threshold for triggering actions
        self.threshold = threshold
        # Boolean to enable/disable explanation
        self.explanation = explanation
    
    def on_llm_start(self, serialized, prompts, run_id, **kwargs):
        # Store input prompt
        self.prompt = prompts[0]
    
    def on_llm_end(self, response, **kwargs):
        # Extract response text from LLMResult object which is the return type for most LLMs
        response_text = response.generations[0][0].text

        # Call trustworthiness score method, and extract the score
        resp = self.tlm.get_trustworthiness_score(self.prompt, response_text)
        score = resp['trustworthiness_score']

        # Log score
        # This can be replaced with any action that the application requires
        if score < self.threshold:
            print(f"[TLM Score]: {score} (Untrustworthy)")
        else:
            print(f"[TLM Score]: {score} (Trustworthy)")

        # Log explanation
        # This can be replaced with any action that the application requires
        if self.explanation and resp.get("log", {}).get("explanation"):
            print(f"[TLM Score Explanation]: {resp['log']['explanation']}")
        elif self.explanation:
            print("[TLM Warning]: Enable `explanation` in TLM client.")

We then instantiate the Cleanlab Studio client -- you can get your free Cleanlab API key [here](https://app.cleanlab.ai/).

We would also instantiate the TLM object that would be used by this callback, and specify that we want explanations in the `options` argument to obtain explanations in the TLM response.

In [8]:
client = Studio(api_key="<cleanlab_api_key>")
tlm = client.TLM(options={'log':['explanation']}) 

Then, we will define an instance of the call function with the TLM object, setting `explanation=True`. Currently, the callback would print the trustworthiness score and explanation, but you can modify the action basis your requirements.

We'll attach this callback to the LLM so that it triggers when the LLM is called. 
You can attach the callback to multiple LLMs, agents, and other objects that generates a response.

In [9]:
trustworthiness_callback = TrustworthinessScoreCallback(tlm, explanation=True)
model.callbacks = [trustworthiness_callback]

Let's reset the memory of the assistant and ask the same questions

In [10]:
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "with-tlm"}}

In [15]:
query = "Which is the biggest lake in California?"
input_messages = [HumanMessage(query)]

output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

[TLM Score]: 0.6246266556912332 (Untrustworthy)
[TLM Score Explanation]: The proposed answer states that Lake Tahoe is the biggest lake in California. However, this is incorrect because the largest lake entirely within California is actually the Salton Sea. Lake Tahoe, while it is a significant and well-known lake, is not the largest lake in California when considering the size of lakes that are entirely within the state's borders. Therefore, the answer provided is factually incorrect. 
This response is untrustworthy due to lack of consistency in possible responses from the model. Here's one inconsistent alternate response that the model considered (which may not be accurate either): 
The biggest lake in California is the Salton Sea.
================================== Ai Message ==================================

The biggest lake in California is Lake Tahoe. It is known for its stunning clear waters and is located in the Sierra Nevada mountain range, straddling the border between Cali

In [16]:
query = "How deep is it?"
input_messages = [HumanMessage(query)]

output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

[TLM Score]: 0.9875591947901597 (Trustworthy)
[TLM Score Explanation]: Did not find a reason to doubt trustworthiness.
================================== Ai Message ==================================

Lake Tahoe has a maximum depth of about 1,645 feet (501 meters), making it the second-deepest lake in the United States, after Crater Lake in Oregon. Its depth contributes to its clarity and unique environmental conditions.


As observed above, hallucinations in LLM outputs can be automatically detected using TLM's trustworthiness score

TLM also provides explanations in complement to the trustworthiness score. You can disable it by setting `TrustworthinessScoreCallback(tlm, explanation=False)`.